In [ ]:
# 🔧 COMPLETE ALEXNET CROSS-VALIDATION TRAINING PIPELINE
# Run this cell to automatically: Install packages → Download data → Train → Evaluate → Download results

# ===== STEP 1: INSTALL PACKAGES =====
import subprocess
import sys
import os

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install required packages
packages = [
    "torch", "torchvision", "torchaudio", 
    "opencv-python-headless", "pillow", "pandas", "tqdm", 
    "gdown", "albumentations", "matplotlib", "seaborn", "scikit-learn"
]

print("📦 Installing required packages...")
for package in packages:
    try:
        install_package(package)
    except:
        pass

# ===== STEP 2: IMPORT LIBRARIES =====
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import json
import zipfile
import gdown

print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device('cuda')
else:
    print("⚠️ Using CPU - training will be slower")
    device = torch.device('cpu')

# ===== STEP 3: DOWNLOAD DATASET =====
DATASET_ID = "1ZAgz5u64i3LDbwMFpBXjzsKt6FrhNGdW"
DATASET_ZIP = "cropped_dataset_4k_face.zip"
EXTRACT_PATH = "data"

print("📥 Downloading dog emotion dataset...")
if not os.path.exists(DATASET_ZIP):
    gdown.download(f"https://drive.google.com/uc?id={DATASET_ID}", DATASET_ZIP, quiet=False)
    print(f"✅ Dataset downloaded: {DATASET_ZIP}")
else:
    print(f"✅ Dataset already exists: {DATASET_ZIP}")

# Extract dataset
if not os.path.exists(EXTRACT_PATH):
    print("📂 Extracting dataset...")
    with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)
    print("✅ Dataset extracted successfully")

# Dataset paths
data_root = os.path.join(EXTRACT_PATH, "cropped_dataset_4k_face", "Dog Emotion")
labels_csv = os.path.join(data_root, "labels.csv")

print(f"\n📂 Dataset structure:")
emotions = [d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))]
print(f"   Emotion classes: {emotions}")

for emotion in emotions:
    emotion_path = os.path.join(data_root, emotion)
    if os.path.isdir(emotion_path):
        count = len([f for f in os.listdir(emotion_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"     {emotion}: {count} images")

# ===== STEP 4: PREPARE DATASET =====
class DogEmotionDataset(Dataset):
    def __init__(self, root, labels_csv, transform=None):
        self.root = root
        df = pd.read_csv(labels_csv)
        self.items = df[['filename', 'label']].values
        unique_labels = sorted(df['label'].unique())
        self.label2index = {name: i for i, name in enumerate(unique_labels)}
        self.index2label = {i: name for name, i in self.label2index.items()}
        self.transform = transform
        print(f"📊 Dataset: {len(self.items)} samples")
        print(f"🏷️  Classes: {list(self.label2index.keys())}")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        fn, label_str = self.items[idx]
        label_idx = self.label2index[label_str]
        img_path = os.path.join(self.root, label_str, fn)
        
        try:
            img = Image.open(img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, label_idx
        except Exception as e:
            # Fallback for corrupted images
            img = Image.new('RGB', (224, 224), (0, 0, 0))
            if self.transform:
                img = self.transform(img)
            return img, label_idx

# Create transforms for AlexNet (224x224 ImageNet standard)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create dataset
dataset = DogEmotionDataset(data_root, labels_csv, train_transform)
NUM_CLASSES = len(dataset.label2index)
EMOTION_CLASSES = list(dataset.label2index.keys())

print(f"\n✅ Dataset ready:")
print(f"   Total samples: {len(dataset)}")
print(f"   Number of classes: {NUM_CLASSES}")
print(f"   Emotion classes: {EMOTION_CLASSES}")

# ===== STEP 5: CROSS-VALIDATION SETUP =====
K_FOLDS = 5
EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 1e-4

print(f"🔄 Cross-Validation Configuration:")
print(f"   K-Folds: {K_FOLDS}")
print(f"   Epochs per fold: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Learning rate: {LEARNING_RATE}")

# Prepare labels for stratified split
labels = [dataset.label2index[item[1]] for item in dataset.items]
labels = np.array(labels)

# Create stratified K-fold
kfold = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

print(f"\n📊 Class distribution:")
unique, counts = np.unique(labels, return_counts=True)
for i, (class_idx, count) in enumerate(zip(unique, counts)):
    class_name = EMOTION_CLASSES[class_idx]
    print(f"   {class_name}: {count} samples ({count/len(labels)*100:.1f}%)")

# Results storage
cv_results = {
    'fold_accuracies': [],
    'fold_losses': [],
    'fold_train_histories': [],
    'fold_val_histories': [],
    'fold_predictions': [],
    'fold_true_labels': [],
    'models': []
}

# ===== STEP 6: TRAINING FUNCTIONS =====
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train model for one epoch"""
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(dataloader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    accuracy = correct / total
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy

def evaluate_model(model, dataloader, criterion, device):
    """Evaluate model"""
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = correct / total
    avg_loss = total_loss / len(dataloader)
    return avg_loss, accuracy, all_predictions, all_labels

def create_model():
    """Create AlexNet model"""
    model = models.alexnet(pretrained=True)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, NUM_CLASSES)
    return model

# ===== STEP 7: CROSS-VALIDATION TRAINING =====
os.makedirs("cv_checkpoints", exist_ok=True)

print("🚀 Starting 5-Fold Cross-Validation Training")
print("="*70)

for fold, (train_idx, val_idx) in enumerate(kfold.split(np.arange(len(dataset)), labels)):
    print(f"\n🔄 FOLD {fold + 1}/{K_FOLDS}")
    print("-" * 50)
    
    # Create data samplers
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    
    # Create data loaders
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=2)
    val_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=val_sampler, num_workers=2)
    
    # Create model for this fold
    model = create_model()
    model.to(device)
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    
    # Training history for this fold
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    best_val_acc = 0.0
    
    print(f"📊 Fold {fold + 1} - Train: {len(train_idx)} samples, Val: {len(val_idx)} samples")
    
    # Training loop
    start_time = time.time()
    for epoch in range(EPOCHS):
        # Training
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        
        # Validation
        val_loss, val_acc, _, _ = evaluate_model(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        # Scheduler step
        scheduler.step()
        
        # Save best model for this fold
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_accuracy': val_acc,
                'fold': fold
            }, f"cv_checkpoints/best_model_fold_{fold + 1}.pth")
        
        # Progress update every 10 epochs
        if (epoch + 1) % 10 == 0:
            elapsed = time.time() - start_time
            eta = elapsed * (EPOCHS - epoch - 1) / (epoch + 1)
            print(f"  Epoch {epoch+1:2d}/{EPOCHS} | "
                  f"Train: {train_acc:.4f} | Val: {val_acc:.4f} | "
                  f"Time: {elapsed/60:.1f}m | ETA: {eta/60:.1f}m")
    
    # Final evaluation on validation set
    model.load_state_dict(torch.load(f"cv_checkpoints/best_model_fold_{fold + 1}.pth")['model_state_dict'])
    final_val_loss, final_val_acc, val_predictions, val_true_labels = evaluate_model(
        model, val_loader, criterion, device
    )
    
    # Store results
    cv_results['fold_accuracies'].append(final_val_acc)
    cv_results['fold_losses'].append(final_val_loss)
    cv_results['fold_train_histories'].append({'loss': train_losses, 'accuracy': train_accuracies})
    cv_results['fold_val_histories'].append({'loss': val_losses, 'accuracy': val_accuracies})
    cv_results['fold_predictions'].append(val_predictions)
    cv_results['fold_true_labels'].append(val_true_labels)
    cv_results['models'].append(f"cv_checkpoints/best_model_fold_{fold + 1}.pth")
    
    print(f"✅ Fold {fold + 1} completed - Best Val Accuracy: {final_val_acc:.4f}")

# Calculate cross-validation statistics
mean_accuracy = np.mean(cv_results['fold_accuracies'])
std_accuracy = np.std(cv_results['fold_accuracies'])
mean_loss = np.mean(cv_results['fold_losses'])

print(f"\n🎉 CROSS-VALIDATION COMPLETED!")
print("="*70)
print(f"📊 Results Summary:")
print(f"   Mean Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"   Mean Loss: {mean_loss:.4f}")
print(f"   Accuracy Range: {min(cv_results['fold_accuracies']):.4f} - {max(cv_results['fold_accuracies']):.4f}")

for fold, acc in enumerate(cv_results['fold_accuracies']):
    print(f"   Fold {fold + 1}: {acc:.4f}")

# ===== STEP 8: VISUALIZATION =====
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Training curves for each fold
ax1 = axes[0, 0]
for fold in range(K_FOLDS):
    epochs = range(1, EPOCHS + 1)
    ax1.plot(epochs, cv_results['fold_train_histories'][fold]['accuracy'], 
             label=f'Fold {fold+1}', alpha=0.7)
ax1.set_title('Training Accuracy by Fold', fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Validation curves for each fold
ax2 = axes[0, 1]
for fold in range(K_FOLDS):
    epochs = range(1, EPOCHS + 1)
    ax2.plot(epochs, cv_results['fold_val_histories'][fold]['accuracy'], 
             label=f'Fold {fold+1}', alpha=0.7)
ax2.set_title('Validation Accuracy by Fold', fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Cross-validation accuracy distribution
ax3 = axes[0, 2]
fold_numbers = range(1, K_FOLDS + 1)
bars = ax3.bar(fold_numbers, cv_results['fold_accuracies'], alpha=0.7, color='skyblue')
ax3.axhline(y=mean_accuracy, color='red', linestyle='--', label=f'Mean: {mean_accuracy:.4f}')
ax3.set_title('Final Accuracy by Fold', fontweight='bold')
ax3.set_xlabel('Fold')
ax3.set_ylabel('Accuracy')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, acc in zip(bars, cv_results['fold_accuracies']):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.005, f'{acc:.3f}',
             ha='center', va='bottom', fontweight='bold')

# 4. Loss curves comparison
ax4 = axes[1, 0]
for fold in range(K_FOLDS):
    epochs = range(1, EPOCHS + 1)
    ax4.plot(epochs, cv_results['fold_train_histories'][fold]['loss'], 
             label=f'Train Fold {fold+1}', alpha=0.5, linestyle='-')
    ax4.plot(epochs, cv_results['fold_val_histories'][fold]['loss'], 
             label=f'Val Fold {fold+1}', alpha=0.5, linestyle='--')
ax4.set_title('Training vs Validation Loss', fontweight='bold')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Loss')
ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax4.grid(True, alpha=0.3)

# 5. Combined confusion matrix
ax5 = axes[1, 1]
all_predictions = np.concatenate(cv_results['fold_predictions'])
all_true_labels = np.concatenate(cv_results['fold_true_labels'])
cm = confusion_matrix(all_true_labels, all_predictions)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues', 
            xticklabels=EMOTION_CLASSES, yticklabels=EMOTION_CLASSES, ax=ax5)
ax5.set_title('Normalized Confusion Matrix (All Folds)', fontweight='bold')
ax5.set_xlabel('Predicted')
ax5.set_ylabel('True')

# 6. Accuracy statistics
ax6 = axes[1, 2]
ax6.text(0.1, 0.9, f'AlexNet Cross-Validation Results', fontweight='bold', fontsize=14, transform=ax6.transAxes)
ax6.text(0.1, 0.8, f'Mean Accuracy: {mean_accuracy:.4f}', fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.7, f'Std Accuracy: {std_accuracy:.4f}', fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.6, f'95% CI: [{mean_accuracy - 1.96*std_accuracy:.4f}, {mean_accuracy + 1.96*std_accuracy:.4f}]', 
         fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.5, f'Min Accuracy: {min(cv_results["fold_accuracies"]):.4f}', fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.4, f'Max Accuracy: {max(cv_results["fold_accuracies"]):.4f}', fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.3, f'Epochs per fold: {EPOCHS}', fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.2, f'Dataset size: {len(dataset)} images', fontsize=12, transform=ax6.transAxes)
ax6.text(0.1, 0.1, f'Classes: {NUM_CLASSES} emotions', fontsize=12, transform=ax6.transAxes)
ax6.set_xlim(0, 1)
ax6.set_ylim(0, 1)
ax6.axis('off')

plt.tight_layout()
plt.show()

# Print detailed classification report
print("\n📋 DETAILED CLASSIFICATION REPORT")
print("="*60)
report = classification_report(all_true_labels, all_predictions, 
                             target_names=EMOTION_CLASSES, digits=4)
print(report)

# ===== STEP 9: SAVE RESULTS =====
results_summary = {
    'experiment_info': {
        'model': 'AlexNet',
        'epochs_per_fold': EPOCHS,
        'k_folds': K_FOLDS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE,
        'dataset_size': len(dataset),
        'num_classes': NUM_CLASSES,
        'emotion_classes': EMOTION_CLASSES
    },
    'cross_validation_results': {
        'mean_accuracy': float(mean_accuracy),
        'std_accuracy': float(std_accuracy),
        'fold_accuracies': [float(acc) for acc in cv_results['fold_accuracies']],
        'fold_losses': [float(loss) for loss in cv_results['fold_losses']],
        'confidence_interval_95': [
            float(mean_accuracy - 1.96*std_accuracy), 
            float(mean_accuracy + 1.96*std_accuracy)
        ]
    },
    'classification_metrics': {
        'confusion_matrix': cm.tolist(),
        'classification_report': report
    },
    'model_paths': cv_results['models']
}

# Save results
with open('alexnet_cv_results_summary.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("✅ Results saved to alexnet_cv_results_summary.json")

# ===== STEP 10: DOWNLOAD RESULTS =====
try:
    from google.colab import files
    
    print("📦 Downloading AlexNet cross-validation results...")
    
    # Download the best model from best performing fold
    best_fold_idx = np.argmax(cv_results['fold_accuracies'])
    best_model_path = f"cv_checkpoints/best_model_fold_{best_fold_idx + 1}.pth"
    
    print(f"🏆 Best model: Fold {best_fold_idx + 1} (Accuracy: {cv_results['fold_accuracies'][best_fold_idx]:.4f})")
    files.download(best_model_path)
    files.download('alexnet_cv_results_summary.json')
    
    # Create and download a zip file with all models
    with zipfile.ZipFile('alexnet_all_cv_models.zip', 'w') as zipf:
        for i in range(K_FOLDS):
            model_path = f"cv_checkpoints/best_model_fold_{i + 1}.pth"
            if os.path.exists(model_path):
                zipf.write(model_path, f"alexnet_fold_{i + 1}_model.pth")
        zipf.write('alexnet_cv_results_summary.json', 'alexnet_cv_results_summary.json')
    
    files.download('alexnet_all_cv_models.zip')
    
    print("✅ Download completed! Files downloaded:")
    print(f"   📄 {best_model_path} - Best performing AlexNet model")
    print(f"   📄 alexnet_cv_results_summary.json - Complete results summary")
    print(f"   📦 alexnet_all_cv_models.zip - All 5 fold models + results")

except ImportError:
    print("💾 Running locally - models saved in cv_checkpoints/ directory")
    print("📋 Results summary saved in alexnet_cv_results_summary.json")

print(f"\n🎯 ALEXNET USAGE INSTRUCTIONS:")
print(f"1. Load the best model: {best_model_path}")
print(f"2. Use AlexNet architecture with {NUM_CLASSES} classes")
print(f"3. Input size: 224x224 pixels")
print(f"4. Classes: {EMOTION_CLASSES}")
print(f"5. Expected accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")

# Final summary
print(f"\n" + "="*70)
print(f"🎉 ALEXNET CROSS-VALIDATION TRAINING COMPLETED!")
print(f"📊 Final Results:")
print(f"   ✅ Mean CV Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"   ✅ Best Fold Accuracy: {max(cv_results['fold_accuracies']):.4f}")
print(f"   ✅ Total Training Time: ~{EPOCHS * K_FOLDS / 10:.0f} hours (estimated)")
print(f"   ✅ Models Trained: {K_FOLDS} AlexNet models")
print(f"   ✅ Robust Evaluation: {K_FOLDS}-fold cross-validation")
print(f"="*70)
